In [2]:
import pandas as pd
import numpy as np
import json
import math

import sys
import os

# Add the directory to the Python path
sys.path.append(os.path.abspath('../'))


from call_function import all_similarity, search_title

c:\Users\Asus\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
df_komikcast=pd.read_csv("../data/komikcast.csv")
# Load the JSON file
with open("../data/consine_decription_similarity.json", "r") as infile:
    consine_decription_similarity = json.load(infile)
    
# Inverse: Convert lists back to NumPy arrays
consine_decription_similarity = {
    key: np.array(value) for key, value in consine_decription_similarity.items()
}

consine_sim_indo_komikcast = consine_decription_similarity['komikcast']

# Function

In [4]:
title = 'Kagurabachi'
title = search_title(title, df_komikcast)

In [5]:
similarity = all_similarity(title, df_komikcast, consine_sim_indo_komikcast)

In [6]:
def rank_avg(value, data, order=0):
  if not isinstance(data, np.ndarray):
    data = np.array(data)  # Convert data to NumPy array for efficient operations
  ranks = np.argsort(np.argsort(-data)) + 1 if order == 0 else np.argsort(data) + 1
  return np.mean(ranks[data == value])

In [7]:
def moora(df, criteria): 
    ## Denominator
    denominator = [0] * len(criteria)
    denominator = {c: 0 for c in criteria}
    for c, i in criteria.items():
        for _, row in df.iterrows():
            denominator[c] += row[c]**2
        denominator[c] = math.sqrt(denominator[c])
    denominator_json = {'id' : 0,'title' : 'DENOMINATOR'}
    
    for c, i in criteria.items():
        denominator_json[c] = denominator[c]
    
    ## Normalize
    ri_result = []
    for j, row in df.iterrows():
        row_dict = {'id': row['id']}
        for c, i in criteria.items():
            row_dict[f"ri_{c}"] = row[c] / denominator[c]
            # row_dict[c] = row[c] / denominator[c]
        ri_result.append(row_dict)
    
    ## Weighting
    weight_ri = []
    weight_np = [i[1]+1 for c, i in criteria.items()]
    weight_json = {c: ((i[1]+1)/math.fsum(weight_np)) for c, i in criteria.items()}
    for j, row in df.iterrows():
        row_dict = {'id': row['id']}
        matching_item = next((item for item in ri_result if item['id'] == row['id']), None)
        if matching_item:
            for c, i in criteria.items():
                row_dict[f"ri_weight_{c}"] = matching_item.get(f"ri_{c}", 0) * weight_json[c]
                # row_dict[c] = matching_item.get(c, 0) * weight_json[c]
        weight_ri.append(row_dict)
    
    ## Declaration S+1 & S-i
    si = []
    for row in weight_ri:
        Splus = []
        Smin = []
        for c, i in criteria.items():
            if i[0] == 1:
                Splus.append(row.get(f"ri_weight_{c}", 0))
            elif i[0] == 0:
                Smin.append(row.get(f"ri_weight_{c}", 0))
            else:
                pass
        row_dict = {'id': row['id'], 'S+i': math.fsum(Splus), 'S-i': math.fsum(Smin)}
        si.append(row_dict)
    
    ## Qi
    qi = []
    for row in si:
        qi.append({'id': row['id'],'Qi': row['S+i'] - row['S-i']})
    
    ## Ranking
    rangking = []
    for row in qi:
        qi_values = [item['Qi'] for item in qi]
        rangking.append({'id': row['id'],'rank': rank_avg(row['Qi'], qi_values)})

    df_ri = pd.DataFrame(ri_result)
    df_Weight_ri = pd.DataFrame(weight_ri)
    df_si = pd.DataFrame(si)
    df_qi = pd.DataFrame(qi)
    df_ranking = pd.DataFrame(rangking)
    # df_combine = pd.merge(df, df_ri, df_Weight_ri, df_si, df_qi, df_ranking, on='id')
    df_combine = df.merge(df_ri, on='id').merge(df_Weight_ri, on='id').merge(df_si, on='id').merge(df_qi, on='id').merge(df_ranking, on='id')
    df_result = df_combine.sort_values(by=['rank'], ascending=True)
    
    return df_result

def copras(df, criteria):
    return

def vikor(df, criteria):
    return

def recommendation(df, criteria):
    criteriaValues = list(criteria.keys())
    df = df.sort_values(by=criteriaValues, ascending=False)
    df = df[1:101][['id', 'title']+criteriaValues]
    df = moora(df, criteria) # 1: S+i & 0: s-i
    return df

""" Criteria Explanation.
{keys : [
        0: array pertama untuk menentukan ini S+i/S-i pada bagian "Si"| S+i: semakin tinggi semakin bagus, S-i: semakin rendah semakin bagus,
        1: array kedua untuk menentukan bobot dibagian "Weighting"
    ]
}
"""
criteria = {'image_similarity': [1, 0], 'genre_similarity': [1, 0], 'description_similarity': [1, 0], 'rate': [1, 3]}
df = recommendation(similarity, criteria)
df[:20]

,id,title,image_similarity,genre_similarity,description_similarity,rate,ri_image_similarity,ri_genre_similarity,ri_description_similarity,ri_rate,ri_weight_image_similarity,ri_weight_genre_similarity,ri_weight_description_similarity,ri_weight_rate,S+i,S-i,Qi,rank
24,3634,The Invincible Ancestor In Modern World,0.687500,40.0,0.151114,7.00,0.100895,0.100125,0.470357,0.096775,0.014414,0.014304,0.067194,0.055300,0.151211,0.0,0.151211,1.0
49,5744,Xianzun System in the City,0.671875,20.0,0.127850,7.00,0.098602,0.050063,0.397945,0.096775,0.014086,0.007152,0.056849,0.055300,0.133387,0.0,0.133387,2.0
77,853,Zombie Apocalypse 82-08,0.656250,80.0,0.074683,7.00,0.096309,0.200250,0.232457,0.096775,0.013758,0.028607,0.033208,0.055300,0.130874,0.0,0.130874,3.0
1,5364,The King of Police,0.734375,20.0,0.086927,7.90,0.107774,0.050063,0.270568,0.109217,0.015396,0.007152,0.038653,0.062410,0.123611,0.0,0.123611,4.0
80,1091,Pieces: The Return to Vengeance,0.656250,60.0,0.057391,7.80,0.096309,0.150188,0.178634,0.107835,0.013758,0.021455,0.025519,0.061620,0.122353,0.0,0.122353,5.0
31,8036,No Game No Life,0.687500,20.0,0.064609,8.14,0.100895,0.050063,0.201102,0.112535,0.014414,0.007152,0.028729,0.064306,0.114600,0.0,0.114600,6.0
22,2952,Island Part 2,0.687500,60.0,0.051681,7.00,0.100895,0.150188,0.160862,0.096775,0.014414,0.021455,0.022980,0.055300,0.114149,0.0,0.114149,7.0
14,8300,Koe no Katachi,0.703125,40.0,0.034692,8.74,0.103188,0.100125,0.107982,0.120830,0.014741,0.014304,0.015426,0.069046,0.113516,0.0,0.113516,8.0
94,2860,Matei Kyoushi to Juuzoku Shoujo no Haitoku Kei...,0.656250,40.0,0.060846,7.00,0.096309,0.100125,0.189389,0.096775,0.013758,0.014304,0.027056,0.055300,0.110417,0.0,0.110417,9.0
4,1318,Youngest Son of the NamGung Clan,0.718750,40.0,0.036061,7.99,0.105481,0.100125,0.112242,0.110461,0.015069,0.014304,0.016035,0.063121,0.108528,0.0,0.108528,10.0


In [9]:
merge_df_moora = pd.merge(df, similarity, on='id')

In [10]:
merge_df_moora

,id,title_x,image_similarity_x,genre_similarity_x,description_similarity_x,rate_x,ri_image_similarity,ri_genre_similarity,ri_description_similarity,ri_rate,...,type,description,genre,author,artist,rate_y,image,released,image_hash,genre_similarity_y
0,3634,The Invincible Ancestor In Modern World,0.687500,40.0,0.151114,7.00,0.100895,0.100125,0.470357,0.096775,...,Manhua,Ye Jinyu mengurung dirinya selama lima ratus t...,"Drama, Fantasy, Martial Arts, Shounen",琪漫文化-小麦,-,7.00,https://komikcast.cz/wp-content/uploads/2022/0...,2022,fcc8c8e4ff9c00c0,40.0
1,5744,Xianzun System in the City,0.671875,20.0,0.127850,7.00,0.098602,0.050063,0.397945,0.096775,...,Manhua,Zhong maadalah seorang mahasiswa biasatibatiba...,"Action, Comedy, Ecchi, Harem, School Life",Man man culture,-,7.00,https://komikcast.cz/wp-content/uploads/2020/0...,2020,6160667fff372200,20.0
2,853,Zombie Apocalypse 82-08,0.656250,80.0,0.074683,7.00,0.096309,0.200250,0.232457,0.096775,...,Manhwa,Virus mematikan yang dimulai dengan itu dan da...,"Action, Drama, Horror, Mature, Seinen, Superna...","Box Office Dal-ah (Author), Gyeong-uh (Artist)",-,7.00,https://komikcast.cz/wp-content/uploads/2023/0...,2023,30202e3e1f1b7f66,80.0
3,5364,The King of Police,0.734375,20.0,0.086927,7.90,0.107774,0.050063,0.270568,0.109217,...,Manhua,Karena panggilan majikan Yin Ziyu kembali ke C...,"Action, Sci-Fi",Long You You x Novel Mars,-,7.90,https://komikcast.cz/wp-content/uploads/2020/0...,2020,7ce0f8de9f090300,20.0
4,1091,Pieces: The Return to Vengeance,0.656250,60.0,0.057391,7.80,0.096309,0.150188,0.178634,0.107835,...,Manga,Square adalah sebuah wilayah yang diselimuti o...,"Action, Drama, Fantasy, Martial Arts, Shounen",YUUYUU Rakuraku,-,7.80,https://komikcast.cz/wp-content/uploads/2024/0...,2023,ffe3e27f3c3c4200,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7401,Engage Princess – Side・By・Side,0.687500,0.0,0.002019,7.00,0.100895,0.000000,0.006286,0.096775,...,Manga,Perayaan Dengeki Bunko ke25 Engage Princess mu...,"Adventure, Comedy, Isekai",Peke,-,7.00,https://komikcast.cz/wp-content/uploads/2019/0...,2019,f8c060fe2e06216f,0.0
96,3845,Stalkers,0.671875,0.0,0.001124,7.01,0.098602,0.000000,0.003499,0.096913,...,Manga,Aku selalu memikirkan tentang orang yang kusuk...,"Comedy, Romance, School Life",Shiori Hanatsuka,-,7.01,https://komikcast.cz/wp-content/uploads/2019/0...,2018,f0f6f4fc8c088080,0.0
97,1884,"Saeki-san to, Hitotsu Yane no Shita: I’ll Have...",0.671875,0.0,0.000571,7.00,0.098602,0.000000,0.001776,0.096775,...,Manga,Memasuki tahun ke2 SMA Yumizuki pindah ke apar...,"Comedy, Romance, School Life, Seinen, Slice of...","Kuyou (Author), Itsutsuse (Artist)",-,7.00,https://komikcast.cz/wp-content/uploads/2022/0...,2017,f3e1eefeeda00000,0.0
98,7705,Ruler of the Three Kingdoms,0.671875,20.0,0.023508,0.00,0.098602,0.050063,0.073170,0.000000,...,Manhua,\n\n Post Views 8,"Action, Historical",?,-,0.00,https://komikcast.cz/wp-content/uploads/2018/0...,?,f0e0e0fd7f96c04f,20.0


In [7]:
df.sort_values(['image_similarity', 'genre_similarity', 'description_similarity', 'rate'], ascending=False, inplace=True)

In [8]:
df

,id,title,image_similarity,genre_similarity,description_similarity,rate,ri_image_similarity,ri_genre_similarity,ri_description_similarity,ri_rate,ri_weight_image_similarity,ri_weight_genre_similarity,ri_weight_description_similarity,ri_weight_rate,S+i,S-i,Qi,rank
0,6726,King of The Soldiers,0.828125,20.0,0.010708,7.00,0.121533,0.050063,0.033329,0.096775,0.017362,0.007152,0.004761,0.055300,0.084575,0.0,0.084575,72.0
1,5364,The King of Police,0.734375,20.0,0.086927,7.90,0.107774,0.050063,0.270568,0.109217,0.015396,0.007152,0.038653,0.062410,0.123611,0.0,0.123611,4.0
2,673,Tensei Goblin da kedo Shitsumon aru?,0.734375,0.0,0.008309,7.00,0.107774,0.000000,0.025862,0.096775,0.015396,0.000000,0.003695,0.055300,0.074391,0.0,0.074391,92.0
3,6731,I Shall Seal the Heavens,0.718750,60.0,0.002122,7.00,0.105481,0.150188,0.006605,0.096775,0.015069,0.021455,0.000944,0.055300,0.092768,0.0,0.092768,46.0
4,1318,Youngest Son of the NamGung Clan,0.718750,40.0,0.036061,7.99,0.105481,0.100125,0.112242,0.110461,0.015069,0.014304,0.016035,0.063121,0.108528,0.0,0.108528,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1140,Saihate no Paladin,0.656250,40.0,0.047700,7.42,0.096309,0.100125,0.148471,0.102581,0.013758,0.014304,0.021210,0.058618,0.107890,0.0,0.107890,12.0
96,8210,Assassination Classroom,0.656250,40.0,0.032431,8.33,0.096309,0.100125,0.100944,0.115162,0.013758,0.014304,0.014421,0.065807,0.108289,0.0,0.108289,11.0
97,1530,The World’s Only Job Changer,0.656250,40.0,0.023536,7.20,0.096309,0.100125,0.073258,0.099540,0.013758,0.014304,0.010465,0.056880,0.095407,0.0,0.095407,36.0
98,200,My Wife Is From a Thousand Years Ago,0.656250,40.0,0.020449,7.00,0.096309,0.100125,0.063649,0.096775,0.013758,0.014304,0.009093,0.055300,0.092455,0.0,0.092455,48.0


In [9]:
df.to_csv('test.csv', index=False)

In [10]:
criteria = {'image_similarity': [1, 0], 'genre_similarity': [1, 0], 'description_similarity': [1, 0], 'rate': [1, 3]}
criteria = sorted(criteria.items(), key=lambda x: x[1], reverse=True)
c = [c for c,i in criteria]
c

['rate', 'image_similarity', 'genre_similarity', 'description_similarity']

In [11]:
criteria = {'image_similarity': [1, 0], 'genre_similarity': [1, 10], 'description_similarity': [1, 0], 'rate': [1, 0]}
w_json = []
weight_np = [i[1]+1 for c, i in criteria.items()]
weight_json = {c: ((i[1]+1)/math.fsum(weight_np)) for c, i in criteria.items()}
ws = [((i[1]+1)/math.fsum(weight_np)) for c, i in criteria.items()]
print(weight_np)
print(weight_json)
print(math.fsum(ws))

[1, 11, 1, 1]
{'image_similarity': 0.07142857142857142, 'genre_similarity': 0.7857142857142857, 'description_similarity': 0.07142857142857142, 'rate': 0.07142857142857142}
1.0
